In [1]:
import glob
import numpy as np
import pandas as pd

from datetime import datetime
from os import listdir
from pptx import Presentation 
from pptx.chart.data import CategoryChartData
from pptx.dml.color import RGBColor
from pptx.enum.chart import XL_CHART_TYPE, XL_LEGEND_POSITION, XL_DATA_LABEL_POSITION, XL_MARKER_STYLE, XL_TICK_MARK
from pptx.enum.dml import MSO_LINE_DASH_STYLE, MSO_THEME_COLOR
from pptx.util import Inches, Pt


In [42]:
def read_data(file):
    filepaths = glob.glob("../data/tidy/" + file + "/" + file + "_daily_doses*.csv")
    df = pd.concat(map(lambda x: pd.read_csv(x, index_col=0, parse_dates=True), filepaths), sort=False)
    df.sort_index(inplace=True)
    return df

def prep_df(df):
    df = df.resample("MS").sum()
    df["FY"] = (df.index + pd.DateOffset(months=6)).strftime('%Y')
    df["idx"] = pd.to_datetime("2019-" + (df.index + pd.DateOffset(months=-6)).strftime('%m-%d')) + pd.DateOffset(months=6)
    df_pvt = df['2016-07-01':].pivot(index="idx", columns="FY", values="DOSES")

    return df_pvt


In [3]:
def add_forecast_slide(p, df, med):
    blank_slide_layout = p.slide_layouts[6]
    slide = p.slides.add_slide(blank_slide_layout)

    chart_data = CategoryChartData()
    chart_data.categories = df.index
    chart_data.add_series("Actual", df["Actual"])
    chart_data.add_series("Forecast", df["Forecast"])
    chart_data.add_series("Upper", df["Upper"])
    chart_data.add_series("Lower", df["Lower"])
    
# use this if want to show separate use on inpt/outpt basis    
#     if "Outpatient" in df.columns:
#         chart_data.add_series("Outpatient", df["Outpatient"])
#         chart_data.add_series("Inpatient", df["Inpatient"])

    x, y, cx, cy = Inches(1), Inches(1), Inches(8), Inches(6)
    chart = slide.shapes.add_chart(XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data).chart
    chart = format_graph(chart, med)
    return p

def format_graph(chart, med, font_nm=None, ax_bright=0.5):
    category_axis = format_axis_title(chart.category_axis, "Month", font_nm, ax_bright)
    category_axis = format_axis_tick_labels(category_axis, font_nm, ax_bright, "mmm yy")
    category_axis = format_axis(category_axis, ax_bright)
#     category_axis.maximum_scale = len(df_combined) + 1
#     df_combined.asfreq("MS").index[-1] + pd.DateOffset(months=1)

    value_axis = format_axis_title(chart.value_axis, "Doses per month", font_nm, ax_bright)
    value_axis = format_axis_tick_labels(value_axis, font_nm, ax_bright, "#,##0")
    value_axis = format_axis(value_axis, ax_bright)

    if med == "ivig":
        med_title = med.upper()
    elif med == "acetaminophen-iv":
        med_title = "Acetaminophen IV "
    elif med == "bupivacaine-liposomal":
        med_title = "Bupivacaine (liposomal) "
    else:
        med_title = med.title()
    
    chart.has_title = True
    chart.chart_title.text_frame.text = med_title + " forecast"
    chart.chart_title.text_frame.paragraphs[0].font.name = font_nm
    chart.chart_title.text_frame.paragraphs[0].font.size = Pt(24)
    chart.chart_title.text_frame.paragraphs[0].font.bold = False
    chart.chart_title.text_frame.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    chart.chart_title.text_frame.paragraphs[0].font.color.brightness = 0.25

    chart.has_legend = False
    # chart.legend.include_in_layout = False
    # chart.legend.position = XL_LEGEND_POSITION.TOP

    # format Actual line
    col_actual = MSO_THEME_COLOR.ACCENT_1
    chart.series[0].format.line.width = Pt(3.5)
    chart.series[0].format.line.color.theme_color = col_actual

    i = len(chart.series[0].points) - 13
    chart = format_marker(chart, 0, i, font_nm, MSO_THEME_COLOR.BACKGROUND_1, col_actual)

    # format Forecast line
    brght_forecast = 0.4

    chart.series[1].format.line.dash_style = MSO_LINE_DASH_STYLE.DASH
    chart.series[1].format.line.width = Pt(2.5)
    chart.series[1].format.line.color.theme_color = col_actual
    chart.series[1].format.line.color.brightness = brght_forecast

    j = len(chart.series[1].points) - 1
    chart = format_marker(chart, 1, j, font_nm, col_actual, col_actual, brght_forecast)

    col_ci = MSO_THEME_COLOR.BACKGROUND_1
    brght_ci = -0.05
    line_ci = Pt(1.5)
    chart.series[2].format.line.width = line_ci
    chart.series[2].format.line.color.theme_color = col_ci
    chart.series[2].format.line.color.brightness = brght_ci
    chart.series[3].format.line.width = line_ci
    chart.series[3].format.line.color.theme_color = col_ci
    chart.series[3].format.line.color.brightness = brght_ci
   
    if len(chart.series) > 4:
        chart.series[4].format.line.width = line_ci
        chart.series[4].format.line.color.theme_color = MSO_THEME_COLOR.ACCENT_3
        chart.series[4].format.line.color.brightness = brght_forecast

        chart.series[5].format.line.width = line_ci
        chart.series[5].format.line.color.theme_color = MSO_THEME_COLOR.ACCENT_2
        chart.series[5].format.line.color.brightness = brght_forecast

    return chart

def format_axis(axis, ax_bright):
    axis.format.line.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.format.line.color.brightness = ax_bright
    axis.has_major_gridlines = False
    axis.has_minor_gridlines = False
    return axis

def format_axis_title(axis, title_text, font_nm, ax_bright, has_title = True):
    axis.has_title = has_title
    axis.axis_title.text_frame.text = title_text
    axis.axis_title.text_frame.paragraphs[0].font.name = font_nm
    axis.axis_title.text_frame.paragraphs[0].font.size = Pt(18)
    axis.axis_title.text_frame.paragraphs[0].font.bold = False
    axis.axis_title.text_frame.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.axis_title.text_frame.paragraphs[0].font.color.brightness = ax_bright
    return axis

def format_axis_tick_labels(axis, font_nm, ax_bright, num_fmt):
    axis.tick_labels.font.name = font_nm
    axis.tick_labels.font.size = Pt(16)
    axis.tick_labels.font_bold = False
    axis.tick_labels.font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.tick_labels.font.color.brightness = ax_bright
    axis.tick_labels.number_format = num_fmt
    return axis

def format_marker(chart, i, j, font_nm, col_mark, col_line, brght=0):
    if i == 0:
        mrk_sz = 7
    else: 
        mrk_sz = 5
    
#     chart.series[i].data_labels.show_value = True
    chart.series[i].data_labels.number_format = "#,##0"
    chart.series[i].data_labels.font.name = font_nm
    chart.series[i].data_labels.font.size = Pt(16)
    chart.series[i].data_labels.font.bold = False
    chart.series[i].data_labels.position = XL_DATA_LABEL_POSITION.ABOVE
    chart.series[i].points[j].marker.style = XL_MARKER_STYLE.CIRCLE
    chart.series[i].points[j].marker.size = mrk_sz
    chart.series[i].points[j].marker.format.fill.solid()
    chart.series[i].points[j].marker.format.fill.fore_color.theme_color = col_mark
    chart.series[i].points[j].marker.format.fill.fore_color.brightness = brght
    chart.series[i].points[j].marker.format.line.width = Pt(2.5)
    chart.series[i].points[j].marker.format.line.color.theme_color = col_line
    chart.series[i].points[j].marker.format.line.color.brightness = brght

    return chart

In [4]:
def make_slides(meds, n=12):
    prs = Presentation()
    # title slide
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = "Forecast for Target Medications"
    fcast_start = datetime.now().strftime('%B %Y')
    fcast_end = (datetime.now() + pd.DateOffset(months=11)).strftime('%B %Y')
    subtitle.text = fcast_start + " to " + fcast_end + "\nBrian Gulbis, PharmD, BCPS"

    for i in meds:
        df = make_forecast_df(i)
#         start_month = (datetime.now() + pd.DateOffset(months=-24)).strftime('%Y-%m')
        add_forecast_slide(prs, df, i)

    return prs

In [5]:
meds = ["acetaminophen-iv", "albumin", "sugammadex", "bupivacaine-liposomal", "isoproterenol", "ivig", "calcitonin", 
        "pegfilgrastim", "eculizumab", "ceftaroline", "ceftazidime-avibactam", "ceftolozane-tazobactam", "daptomycin",
        "ertapenem", "meropenem-vaborbactam"]
prs = make_slides(meds)
save_month = (datetime.now() + pd.DateOffset(months=-1)).strftime('%Y-%m')
prs.save('../report/utilization/target_med_forecast_' + save_month + '.pptx')

In [37]:
df = read_data("acetaminophen-iv")
df = df.resample("MS").sum()
df["FY"] = (df.index + pd.DateOffset(months=6)).strftime('%Y')
df["Month"] = df.index.strftime('%B')
df["idx"] = pd.to_datetime("2019-" + (df.index + pd.DateOffset(months=-6)).strftime('%m-%d')) + pd.DateOffset(months=6)
df_pvt = df['2016-07-01':].pivot(index="idx", columns="FY", values="DOSES")

In [55]:
df_pvt[:,:]

TypeError: '(slice(None, None, None), slice(None, None, None))' is an invalid key

In [46]:
for i in range(0, len(df_pvt.columns)):
    print(df_pvt.columns[i])
#     df_pvt[i]

2017


KeyError: 0